In [1]:
import random
import numpy as np
import pathlib
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import time
import sys
import sklearn
from tensorflow import keras
from tensorflow.keras import losses, optimizers
import tensorflow as tf
%matplotlib inline

In [2]:
# 设置参数
data_dir = "pokemen"
train_txt = os.path.join(data_dir, "list_train_shuffle.txt")
val_txt = os.path.join(data_dir, "list_val_shuffle.txt")
train_dir = data_dir
val_dir = data_dir

AUTOTUNE = tf.data.experimental.AUTOTUNE
height = 224
width = 224
channels = 3
BATCH_SIZE = 4
num_classes = 5
base_lr = 1e-2
gamma = 0.1
momentum = 0.9
epochs = 100

### 读图片路径 && 保存路径和标签到文件
1. 读文件夹
2. 遍历文件夹内的所有图形
3. 保存到文件

In [3]:
def read_imageset_to_txt(read_folder, save_path):
    def format_txt_lines(path, index):
        return ''.join(path + " " + str(index) + "\n")
    
    # 输入判断
    if not os.path.exists(read_folder):
        raise ValueError("can not find folder:" + read_folder)
    if os.path.exists(save_path):
        os.remove(save_path)
    
    # 读所有文件夹
    dirs = os.listdir(read_folder)
    dirs = filter(lambda f : os.path.isdir(os.path.join(read_folder, f)), dirs)
    
    for i, sub_dir in enumerate(list(dirs)):
        # 读文件夹图像
        images_path = glob.glob( os.path.join(read_folder, sub_dir, "*.??g") )
        labels = [i] * len(images_path)
        labels_images = list(map(format_txt_lines, images_path, labels))
        with open(save_path, 'a') as f:
            for label in labels_images:
                f.write(label)
        
        print("folder {} have {} images.".format(sub_dir, len(images_path)))

In [4]:
save_path = "./list_train.txt"
read_imageset_to_txt(data_dir, save_path)

folder bulbasaur have 233 images.
folder charmander have 236 images.
folder mewtwo have 237 images.
folder pikachu have 232 images.
folder squirtle have 221 images.


### TF载入图像数据
1. 读入数据集
2. 解析数据

In [5]:
def tf_load_and_prase_data(train_txt, val_txt = None):
    """
        读入数据集，并返回可使用的tf对象
        
        input:
            train_txt: 训练集数据
            val_txt：验证集数据
        
        return：
            train_ds, val_ds
    """
    def read_lines_from_txt(txt):
        lines = []
        with open(txt, 'r') as f:
            lines = f.readlines()
        return lines
    
    def split_dataset(dataset, prop = 0.05):
        train_data = []
        val_data = []
        for data in dataset:
            if random.random() > prop:
                train_data.append(data)
            else:
                val_data.append(data)
        return train_data, val_data
    
    def tf_read_and_prase_data(dataset):
        """
            input:
                dataset: [lines, lines, lines, ...]
            
            output:
                ds
        """
        def preprocess_image(image):
            image = tf.image.decode_jpeg(image, channels = channels)
            image = tf.image.resize(image, [height, width])
            image /= 255.0  # normalize to [0,1] range
            return image

        def load_and_preprocess_image(path):
            image = tf.io.read_file(path)
            return preprocess_image(image)
        
        # 分割数据
        images_path = []
        labels = []
        for data in dataset:
            if len(data.strip().split(" ")[1:]) != 0:
                images_path.append(data.strip().split(" ")[0])
                labels.append(int(data.strip().split(" ")[1]))

        # 读取数据
        path_ds = tf.data.Dataset.from_tensor_slices(images_path)
        image_ds = path_ds.map(load_and_preprocess_image)
        label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(labels, tf.int64))
        
        # 解析数据
        return tf.data.Dataset.zip((image_ds, label_ds))
        
    
    # 输入判断
    if not os.path.exists(train_txt):
        raise ValueError("can not find the file:" + train_txt)             
    if val_txt is not None and not os.path.exists(val_txt):
        raise ValueError("can not find the file:" + val_txt)
    
    
    # 区分 && 打乱数据集
    if val_txt is not None:
        train_path_label = read_lines_from_txt(train_txt)
        val_path_label = read_lines_from_txt(val_txt)
    else:
        train_data = read_lines_from_txt(train_txt)
        train_path_label, val_path_label = split_dataset(train_data)
    
    num_train = len(train_path_label)
    num_valid = len(val_path_label)
    print("numbers of train dataset:{}, numbers of val dataset:{}".format(num_train, num_valid))
    
    # 读入 && 解析数据
    train_ds = tf_read_and_prase_data(train_path_label)
    val_ds = tf_read_and_prase_data(val_path_label)
    
    return train_ds, num_train, val_ds, num_valid
    

In [6]:
def tf_load_and_prase_data_v2(train_txt, val_txt = None):
    """
        读入数据集，并返回可使用的tf对象
        
        input:
            train_txt: 训练集数据
            val_txt：验证集数据
        
        return：
            train_ds, val_ds
    """
    def read_lines_from_txt(txt):
        lines = []
        with open(txt, 'r') as f:
            lines = f.readlines()
        return lines
    
    def split_dataset(dataset, prop = 0.05):
        train_data = []
        val_data = []
        for data in dataset:
            if random.random() > prop:
                train_data.append(data)
            else:
                val_data.append(data)
        return train_data, val_data
    
    def tf_read_and_prase_data(dataset):
        """
            input:
                dataset: [lines, lines, lines, ...]
            
            output:
                ds
        """
        def preprocess_image(image):
            image = tf.image.decode_jpeg(image, channels = channels)
            image = tf.image.resize(image, [height, width])
            image /= 255.0  # normalize to [0,1] range
            return image

        def load_and_preprocess_image(path):
            image = tf.io.read_file(path)
            return preprocess_image(image)
        
        # 分割数据
        images_path = []
        labels = []
        for data in dataset:
            if len(data.strip().split(" ")[1:]) != 0:
                images_path.append(data.strip().split(" ")[0])
                labels.append(int(data.strip().split(" ")[1]))

        # 读取数据
        ds = tf.data.Dataset.from_tensor_slices((images_path, labels))
        
        # 解析数据
        return ds
        
    
    # 输入判断
    if not os.path.exists(train_txt):
        raise ValueError("can not find the file:" + train_txt)             
    if val_txt is not None and not os.path.exists(val_txt):
        raise ValueError("can not find the file:" + val_txt)
    
    
    # 区分 && 打乱数据集
    if val_txt is not None:
        train_path_label = read_lines_from_txt(train_txt)
        val_path_label = read_lines_from_txt(val_txt)
    else:
        train_data = read_lines_from_txt(train_txt)
        train_path_label, val_path_label = split_dataset(train_data)
    
    num_train = len(train_path_label)
    num_valid = len(val_path_label)
    print("numbers of train dataset:{}, numbers of val dataset:{}".format(num_train, num_valid))
    
    # 读入 && 解析数据
    train_ds = tf_read_and_prase_data(train_path_label)
    val_ds = tf_read_and_prase_data(val_path_label)
    
    return train_ds, num_train, val_ds, num_valid
    

In [7]:
txt = "./list_train.txt"
train_origin_ds, num_train, val_origin_ds, num_valid = tf_load_and_prase_data_v2(txt)

numbers of train dataset:1091, numbers of val dataset:68


In [12]:
def tf_shuffle_data(dataset, buffer_size, batch_size):
    def normalize(x, mean=tf.constant([0.485, 0.456, 0.406]), std= tf.constant([0.229, 0.224, 0.225])):
        # 标准化
        # x: [224, 224, 3]
        # mean: [224, 224, 3], std: [3]
        x = (x - mean)/std
        return x

    def preprocess(x,y):
        # x: 图片的路径，y：图片的数字编码
        x = tf.io.read_file(x)
        x = tf.image.decode_jpeg(x, channels=3) # RGBA
        x = tf.image.resize(x, [244, 244])

        x = tf.image.random_flip_left_right(x)
        x = tf.image.random_flip_up_down(x)
        x = tf.image.random_crop(x, [224,224,3])

        # x: [0,255]=> -1~1
        x = tf.cast(x, dtype=tf.float32) / 255.
        x = normalize(x)
        y = tf.convert_to_tensor(y)
        y = tf.one_hot(y, depth=5)

        return x, y
    
    ds = dataset.shuffle(buffer_size=buffer_size)
    ds = ds.repeat()
    ds = ds.map(preprocess).batch(batch_size)
    # 当模型在训练的时候，`prefetch` 使数据集在后台取得 batch。
    ds = ds.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)
    return ds

In [14]:
train_ds = tf_shuffle_data(train_origin_ds, num_train, BATCH_SIZE)
val_ds = tf_shuffle_data(val_origin_ds, num_valid, BATCH_SIZE)

In [15]:
# 创建模型
model = keras.models.Sequential([
    keras.layers.Conv2D(64, (3,3), padding = 'same', activation = 'relu', strides = 2, input_shape = (height, width, channels)),
    keras.layers.Conv2D(64, (3,3), padding = 'same', activation = 'relu', strides = 2),
    keras.layers.Conv2D(128, (3,3), padding = 'same', activation = 'relu', strides = 2),
    keras.layers.Conv2D(128, (3,3), padding = 'same', activation = 'relu', strides = 2),
    keras.layers.Conv2D(256, (3,3), padding = 'same', activation = 'relu', strides = 2),
    keras.layers.AveragePooling2D((7,7)),
    keras.layers.Flatten(),
    keras.layers.Dense(num_classes, activation = 'softmax')
])

In [16]:
# 创建优化器等 
model.compile(loss = "categorical_crossentropy",
              opentimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 112, 112, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 64)        36928     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 256)         295168    
_________________________________________________________________
average_pooling2d (AveragePo (None, 1, 1, 256)         0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0

In [17]:
# 训练
history = model.fit_generator(train_ds,
                              steps_per_epoch= num_train // BATCH_SIZE,
                              epochs= epochs,
                              validation_data=val_ds,
                              validation_steps= num_valid // BATCH_SIZE)

Epoch 1/100
272/272 [==============================] - 37s 135ms/step - loss: 1.2501 - accuracy: 0.4899 - val_loss: 0.7201 - val_accuracy: 0.6029
Epoch 2/100
272/272 [==============================] - 32s 117ms/step - loss: 0.9114 - accuracy: 0.7188 - val_loss: 0.7717 - val_accuracy: 0.8824
Epoch 3/100
272/272 [==============================] - 22s 82ms/step - loss: 0.8593 - accuracy: 0.7868 - val_loss: 0.6484 - val_accuracy: 0.8235
Epoch 4/100
272/272 [==============================] - 19s 68ms/step - loss: 0.7415 - accuracy: 0.8199 - val_loss: 0.4608 - val_accuracy: 0.8676
Epoch 5/100
272/272 [==============================] - 18s 68ms/step - loss: 0.7614 - accuracy: 0.8346 - val_loss: 0.3859 - val_accuracy: 0.8382
Epoch 6/100
272/272 [==============================] - 35s 128ms/step - loss: 0.8018 - accuracy: 0.8300 - val_loss: 0.4750 - val_accuracy: 0.8971
Epoch 7/100
272/272 [==============================] - 23s 84ms/step - loss: 0.6677 - accuracy: 0.8410 - val_loss: 0.5093 - val

272/272 [==============================] - 12s 45ms/step - loss: 0.6743 - accuracy: 0.8934 - val_loss: 1.1132 - val_accuracy: 0.8529
Epoch 58/100
272/272 [==============================] - 12s 46ms/step - loss: 0.5410 - accuracy: 0.9081 - val_loss: 0.4612 - val_accuracy: 0.9265
Epoch 59/100
272/272 [==============================] - 12s 46ms/step - loss: 0.6995 - accuracy: 0.8961 - val_loss: 0.8849 - val_accuracy: 0.8971
Epoch 60/100
272/272 [==============================] - 12s 43ms/step - loss: 1.1958 - accuracy: 0.8649 - val_loss: 0.4503 - val_accuracy: 0.9265
Epoch 61/100
272/272 [==============================] - 12s 44ms/step - loss: 0.4674 - accuracy: 0.9053 - val_loss: 0.4870 - val_accuracy: 0.9265
Epoch 62/100
272/272 [==============================] - 12s 43ms/step - loss: 0.6617 - accuracy: 0.9053 - val_loss: 0.4739 - val_accuracy: 0.9118
Epoch 63/100
272/272 [==============================] - 12s 45ms/step - loss: 0.5554 - accuracy: 0.9035 - val_loss: 0.3521 - val_accuracy